In [ ]:
#default_exp data.datasets

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export
from unoai.imports import *
from unoai.data.load import *
from unoai.vision.augmentations import *

In [ ]:
#export
def get_cifar10(ds_dir: str, batch_size: int = None,
                normalize: bool = False, padding: int = None, augmentation: str = None):
    if not os.path.exists(ds_dir):
        os.makedirs(ds_dir)

    img_sz = (32, 32)
    fname_base = "cifar10"
    if normalize:
        fname_base += "-normalized"
    if padding is not None:
        fname_base += f"-{padding}-pad"
    if augmentation is not None:
        fname_base += f"-{augmentation}-aug"

    train_path = os.path.join(ds_dir, f"{fname_base}.train.tfrecords")
    test_path = os.path.join(ds_dir, f"{fname_base}.test.tfrecords")

    def parser_train(tf_example):
        if padding is not None:
            train_img_sz = (32+padding*2, 32+padding*2)
        else:
            train_img_sz = img_sz
        x, y = parse_tf_example_img(tf_example, train_img_sz[0], train_img_sz[1])
        if padding is not None:
            x = tf.image.random_flip_left_right(tf.image.random_crop(x, [img_sz[0], img_sz[1], 3]))
        if augmentation and augmentation == 'cutout':
            x = cutout(x,h=8,w=8)
        return x, y

    def parser_test(tf_example):
        x, y = parse_tf_example_img(tf_example, img_sz[0], img_sz[1])
        return x, y

    if not os.path.exists(train_path) or not os.path.exists(test_path):
        (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
        if normalize:
            X_train_mean = np.mean(X_train, axis=(0,1,2))
            X_train_std = np.std(X_train, axis=(0,1,2))
            X_train = (X_train - X_train_mean) / X_train_std
            X_test = (X_test - X_train_mean) / X_train_std
        if padding is not None:
            pad_fn = lambda x: np.pad(x, [(0, 0), (padding, padding), (padding, padding), (0, 0)], mode='reflect')
            X_train = pad_fn(X_train)

        store_np_imgs_as_tfrecord(train_path, X_train, y_train)
        store_np_imgs_as_tfrecord(test_path, X_test, y_test)

    num_train = 50000

    train = read_tfrecord_as_dataset(ds_path=train_path, parser=parser_train, batch_size=batch_size, shuffle=True, shuffle_buffer_size=num_train)
    test = read_tfrecord_as_dataset(ds_path=test_path, parser=parser_test, batch_size=batch_size, shuffle=False)

    return train, test